In [9]:
openai.api_key = "YOUR_KEY"

In [1]:
from tkinter import Tk, Text, Scrollbar, END, VERTICAL, RIGHT, Y
from tkinter import ttk
from tabulate import tabulate
import re
import tkinter as tk
from tkinter import ttk, messagebox
import openai
import sqlite3
import mysql.connector
import time
import requests

# MySQL connection
db = mysql.connector.connect(
    host='pokedex.coozqw8u3amu.us-east-1.rds.amazonaws.com',
    user='admin',
    password='shadowclone'
)
c = db.cursor()

theme = ''
script = ''

# Set up OpenAI API key
openai.api_key = "YOUR_KEY"


def run_sql_query():
    """
    Executes the SQL query entered in the input field and displays the output in the result area.
    """
    query = input_text.get("1.0", END).strip(
    )  # Get SQL query from input field
    result_text.delete("1.0", END)  # Clear the result area

    try:
        cursor.execute(query)  # Execute SQL query
        if query.lower().startswith("select"):  # Display results only for SELECT queries
            rows = cursor.fetchall()
            headers = [description[0] for description in cursor.description]
            result_table = tabulate(rows, headers=headers, tablefmt="grid")
            # Display the result in tabular format
            result_text.insert("1.0", result_table)
        else:
            connection.commit()  # Commit changes for non-SELECT queries
            result_text.insert("1.0", "Query executed successfully.")
    except Exception as e:
        result_text.insert("1.0", f"Error: {str(e)}")  # Display errors if any


def display_table(data, headers=None):
    """
    Displays the data in a tabular format using the tabulate library.

    :param data: List of tuples representing the rows of the table.
    :param headers: List of column headers (optional).
    """
    if headers:
        print(tabulate(data, headers=headers, tablefmt="grid"))
    else:
        print(tabulate(data, tablefmt="grid"))

headers = ['Column Name', 'Data Type', 'Nullable', 'Key', 'Default Value', 'Extra Details']


def generate_script(theme):
    prompt = (
        f"""Create a Python script to connect to a MySQL database with a theme '{theme}'.
        Use these details to create the cursor -
        # MySQL connection
        db = mysql.connector.connect(
            host='pokedex.coozqw8u3amu.us-east-1.rds.amazonaws.com',
            user='admin',
            password='shadowclone'
        )
        c = db.cursor()
        c.execute(f"DROP DATABASE IF EXISTS {theme}")
        c.execute(f"CREATE DATABASE {theme}")
        c.execute(f"USE {theme}")
        Create 4 kid-friendly tables with relations to each other and all columns used for insert queries. Do not skip the autoincrement columns. Make sure there are atleast 4-5 rows in each table.
        """
    )
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=3800
    )
    return response.choices[0].text


def execute_script(script):
    try:
        exec(script)
        return True
    except Exception as e:
        messagebox.showerror("Error", f"Failed to execute script: {e}")
        return False


def show_tables(db_name):
    try:    

        c.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = c.fetchall()

        table_details = ""
        for table_name in tables:
            table_name = table_name[0]
            table_details += f"Table: {table_name}\n"
            c.execute(f"PRAGMA table_info({table_name});")
            columns = c.fetchall()
            for col in columns:
                table_details += f"  {col[1]} ({col[2]})\n"
            table_details += "\n"

        # conn.close()
        return table_details
    except Exception as e:
        return f"Error fetching table details: {e}"


class SQLTeachingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Teach SQL to Kids")

        self.theme_label = ttk.Label(root, text="Enter a Theme:")
        self.theme_label.pack(pady=5)

        self.theme_entry = ttk.Entry(root, width=40)
        self.theme_entry.pack(pady=5)

        self.generate_button = ttk.Button(
            root, text="Generate and Execute", command=self.generate_and_execute
        )
        self.generate_button.pack(pady=10)

        self.output_text = tk.Text(root, wrap=tk.WORD, width=220, height=70)
        self.output_text.pack(pady=20)

    def generate_and_execute(self):
        global theme
        theme = self.theme_entry.get().strip().lower()
        if not theme:
            messagebox.showwarning("Input Required", "Please enter a theme.")
            return

        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, "Generating script...\n")
        self.root.update()

        global script

        script = generate_script(theme)

        with open("generated_script.py", "w") as f:
            f.write(script)

        self.output_text.insert(
            tk.END, "Script generated successfully. Executing...\n")
        self.root.update()
        
        # time.sleep(10)

        success = execute_script(script)
        time.sleep(10)
        if success:
            # print("Script Executed Successfully. Database and Tables created.")
            self.output_text.insert(
                tk.END, "Script Executed Successfully. Database and Tables created.\n")
            self.root.update()
            time.sleep(2)
            # self.show_next_page()
            c.execute(f"USE {theme}")
            c.execute("SHOW Tables")
            tables = c.fetchall()

            x = [item[0].decode('utf-8') for item in tables]
            f = ''
            for i in x:
                c.execute(f'DESCRIBE {i}')
                x_fetch = c.fetchall()
                # y = [item[0].decode('utf-8') for item in x_fetch]
                # display_table(x_fetch, headers)
                self.output_text.insert(
                    tk.END, f'Table Structure for - {i} \n')
                self.output_text.insert(
                    tk.END, tabulate(x_fetch, headers=headers, tablefmt="grid"))
                self.output_text.insert(
                    tk.END, "\n\n")
                self.root.update()

            self.root.update()
            time.sleep(5)
            self.show_next_page()
        else:
            self.output_text.insert(
                tk.END, "Failed to execute script. Check logs for details.\n"
            )

    def show_next_page(self):
        learning_page = tk.Toplevel(self.root)
        learning_page.title("Start Learning")

        prompt = (
            f"""
            Give me 3-4 very basic SQL coding questions to practice basic SQL concepts to kids on {theme} database.
            """
        )
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=3800
        )
        q =  response.choices[0].text
        print(q)

        time.sleep(5)

        learning_label = ttk.Label(
            learning_page,
            text=(
                f"""
                1. SELECT:
                Relatable English Word: "Choose"
                Example:
                Imagine you have a pokemon with all the Pokémon listed. You want to look at only the names of Pokémon. In SQL, we use SELECT to choose what we want to see.

                -----------------------------------------------------------------------------------------------------------------------------------------------------------

                2. FROM:
                Relatable English Word: "From where?"
                Example:
                When you use SELECT, you need to say where to find the Pokémon data. That’s what FROM is for.

                -----------------------------------------------------------------------------------------------------------------------------------------------------------

                3. WHERE:
                Relatable English Word: "Filter" or "Which one?"
                Example:
                Let’s say you want to see only Fire-type Pokémon. You can use WHERE to choose specific ones.

                -----------------------------------------------------------------------------------------------------------------------------------------------------------

                4. COUNT:
                Relatable English Word: "How many?"
                Example:
                You want to know how many Water-type Pokémon are in your pokemon. Use COUNT to find out.

                -----------------------------------------------------------------------------------------------------------------------------------------------------------

                Few Questions - 

                            {q}

                """
            ),
            font=("Aptos", 14),
            wraplength=1600
        )
        learning_label.pack(pady=10)

        explore_button = ttk.Button(
            learning_page, text="Hands On SQL Console", command=lambda: None
        )
        explore_button.pack(pady=10)

if __name__ == "__main__":
    root = tk.Tk()
    app = SQLTeachingApp(root)
    root.mainloop()

2024-12-13 21:04:45.558 Python[11510:588305] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-13 21:04:45.558 Python[11510:588305] +[IMKInputSession subclass]: chose IMKInputSession_Modern



1. What types of information can be stored in a SQL database for Pokemon? 
2. How would you display the names and types of all Pokemon within a certain evolution level range? 
3. Can you write a query to find the average attack and defense stats for all water-type Pokemon? 
4. How would you update a specific Pokemon's level to one level higher?
